# Python Übung 2: Maschinelles Lernen

##### In dieser Python Übung befassen wir uns mit der Anwendung maschineller Lernverfahren auf Bilddaten. Dazu verwenden wir den MNIST Datensatz. Dies ist ein sehr bekannter Datensatz für das Benchmarking von Deep Learning Algorithmen. Zunächst installieren wir alle benötigten Packages und importieren diese.

##### Falls Python für Sie neu ist, empfehlen wir Ihnen folgendes Tutorial: https://www.w3schools.com/python/

In [ ]:
# Installation notwendiger Packages
!pip install sklearn
!pip install tensorflow
!pip install keras
!pip install matplotlib
!pip install seaborn

In [ ]:
# Import notwendiger Packages
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import KFold

from keras.datasets import mnist
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten
from keras.optimizers import SGD

import numpy as np
from matplotlib import pyplot
import seaborn as sns

##### Der MNIST Datensatz besteht aus 70.000 Bildern der Größe 28x28 Pixel. Die Bilder enthalten dabei die Zahlen von 0 bis 9. Wir laden zunächst den Datensatz und überprüfen die Dimensionen der Daten.

In [ ]:
# Lade den Datensatz
(trainX, trainY), (testX, testY) = mnist.load_data()

# Gebe einen Überblick über die Dimensionen aus
print('Train: X={trainX}, y={trainY}'.format(trainX = trainX.shape, trainY = trainY.shape))
print('Test: X={testX}, y={testY}'.format(testX = testX.shape, testY = testY.shape))

##### Wir visualisieren nun ein paar Trainingsdaten. 

In [ ]:
# Plot der ersten 9 Trainingsbilder
for i in range(9):
    pyplot.subplot(330 + 1 + i)
    pyplot.imshow(trainX[i], cmap = pyplot.get_cmap('gray'))
pyplot.show()

# Teil 1: Support Vector Machine (SVM)

##### Zunächst verwenden wir eine SVM zur Klassifikation der Bilddaten. Da SVMs eindimensionale Inputdaten erwarten, müssen die Bilddaten in diese Form transformiert werden.

In [ ]:
trainX_1d = trainX.reshape((60000, 28*28))
testX_1d = testX.reshape((10000, 28*28))

# Gebe einen Überblick über die Dimensionen aus
print('Train: X={trainX}, y={trainY}'.format(trainX = trainX_1d.shape, trainY = trainY.shape))
print('Test: X={testX}, y={testY}'.format(testX = testX_1d.shape, testY = testY.shape))

##### Die SVM wird zunächst mit den Trainingsdaten trainiert. Zur Verringerung der Rechenzeit beschränken wir die maximale Anzahl an Iterationen. Anschließend wird das Modell dazu verwendet, um die Daten aus dem Hold Out Testdatensatz vorherzusagen.

In [ ]:
# instanziiere eine SVM
svm = SVC(max_iter = 50)

# trainiere die SVM mit den Trainingsdaten
svm.fit(trainX_1d, trainY)

# sage die Testdaten vorher
predY_svm = svm.predict(testX_1d)

##### Mithilfe der Testdaten berechnen wir die Konfusionsmatrix für das Modell.

In [ ]:
# berechne die Konfusionsmatrix
cm = metrics.confusion_matrix(testY, predY_svm)

# plotte die Matrix
pyplot.figure(figsize=(9,9))
sns.heatmap(cm, annot = True, fmt = ".3f", linewidths = .5, square = True, cmap = 'Blues_r');
pyplot.ylabel('Tatsächliche Klasse');
pyplot.xlabel('Ermittelte Klasse');

### Aufgabe 1: Berechnen Sie Accuracy, Precision und Recall des Modells.
##### Hinweis: Es empfielt sich, eine Funktion dafür zu schreiben, da die Metriken im späteren Verlauf wieder benötigt werden. Eine Dokumentation zu den Metriken in sklearn finden Sie hier: https://scikit-learn.org/stable/modules/model_evaluation.html

In [ ]:
# Hier ist Platz für Ihre Lösung

### Aufgabe 2: Welche Parameterwerte hat das SVM Modell standardmäßig?
##### Hinweis: Eine Dokumentation zu SVMs in sklearn finden Sie hier: https://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html

In [ ]:
# Hier ist Platz für Ihre Lösung

### Aufgabe 3: Verwenden Sie nun eine SVM mit linearem Kernel. Vergleichen Sie die Performance des neuen Modells mit dem Standard-Modell.

In [ ]:
# Hier ist Platz für Ihre Lösung

# Teil 2: Multi Layer Perceptron (MLP)

##### Im nächsten Schritt verwenden wir ein Feed-Forward Neuronales Netz. Dabei beschränken wir wieder die Anzahl an Iterationen, um die Rechenzeit zu begrenzen. Das MLP erwartet ebenso wie die SVM eindimensionale Inputs.

In [ ]:
# instanziiere ein neuronales Netz
mlp = MLPClassifier(max_iter = 50, verbose = True)

# trainiere das MLP mit den Trainingsdaten
mlp.fit(trainX_1d, trainY)

# sage die Testdaten vorher
predY_mlp = mlp.predict(testX_1d)

In [ ]:
# berechne die Konfusionsmatrix
cm = metrics.confusion_matrix(testY, predY_mlp)

# plotte die Matrix
pyplot.figure(figsize=(9,9))
sns.heatmap(cm, annot = True, fmt = ".3f", linewidths = .5, square = True, cmap = 'Blues_r');
pyplot.ylabel('Tatsächliche Klasse');
pyplot.xlabel('Ermittelte Klasse');

### Aufgabe 4: Berechnen Sie erneut Accuracy, Precision und Recall des Modells. Welche Parameterwerte hat das neuronale Netz standardmäßig?

##### Hinweis: Eine Dokumentation zu MLPs in sklearn finden Sie hier: https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPClassifier.html

In [ ]:
# Hier ist Platz für Ihre Lösung

### Aufgabe 5: Wie lässt sich die Performance des Modells verbessern? Experimentieren Sie mit den Modellparametern. Welche Parameter haben hierbei einen entscheidenden Einfluss?

In [ ]:
# Hier ist Platz für Ihre Lösung

# Teil 3: Convolutional Neural Network (CNN)

##### Zuletzt schauen wir uns CNNs an. Für ausführlichere Erläuterungen sei auf https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-from-scratch-for-mnist-handwritten-digit-classification/ verwiesen. Wie Sie in der Vorlesung bereits erfahren haben, sind CNNs State of the Art, wenn es um die Klassifikation von Bildern geht. Bevor wir das Klassifikationsmodell erstellen, müssen die MNIST Daten noch transformiert werden. Im Gegensatz zu SVMs und MLPs erwartet ein CNN dreidimensionale Inputs. Ein Eingabebild muss in der Dimension b x h x c übergeben werden, wobei:
##### b: Breite des Bildes (#Pixel)
##### h: Höhe des Bildes (#Pixel)
##### c: Anzahl der Channels, bei Schwarz/Weiß-Bildern: 1, bei Farb-Bildern: 3 (für die Pixelcodierung in rot, grün und blau)

In [ ]:
# transformiere den Datensatz
trainX_cnn = trainX.reshape((trainX.shape[0], 28, 28, 1))
testX_cnn = testX.reshape((testX.shape[0], 28, 28, 1))

##### Die Pixelwerte im Datensatz liegen zwischen 0 (schwarz) und 255 (weiß). Bevor wir das CNN-Modell erstellen, sollte der Datensatz normalisiert werden, z.B. durch Skalierung der Pixelfarbwerte auf das Intervall [0,1].

In [ ]:
# skaliere die Trainingsdaten
trainX_cnn = trainX_cnn.astype('float32')
trainX_cnn = trainX_cnn / 255.0

# skaliere die Testdaten
testX_cnn = testX_cnn.astype('float32')
testX_cnn = testX_cnn / 255.0

##### Außerdem codieren wir den Output des Netzes, so dass das CNN später 10 Outputneuronen besitzt (für jede Zahl von 0-9 eine)

In [ ]:
# Kodierung des Outputs
trainY_cnn = to_categorical(trainY)
testY_cnn = to_categorical(testY)

##### Nun erstellen wir ein CNN mit einem Convolutional Layer, einem Max Pooling Layer und einem Fully Connected Layer. Wir trainieren das Modell mit dem stochastischen Gradientenabstiegsverfahren (SGD).

In [ ]:
# erstelle ein CNN-Modell
def create_cnn():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation = 'relu', kernel_initializer = 'he_uniform', 
                     input_shape = (28, 28, 1)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(100, activation = 'relu', kernel_initializer = 'he_uniform'))
    model.add(Dense(10, activation = 'softmax'))
    
    # kompiliere das Modell
    opt = SGD(lr = 0.01, momentum = 0.9)
    model.compile(optimizer = opt, loss = 'categorical_crossentropy', metrics = ['accuracy'])
    return model

### Aufgabe 6: Führen Sie k-Fold Cross Validation (k = 4) mit dem oben definierten CNN-Modell durch. Greifen Sie dabei auf die Funktion create_cnn zurück. Wie robust ist das Modell?
##### Hinweis: Hierfür müssen Sie nur auf die Trainingsdaten zurückgreifen. Sie können die KFold Funktion von sklearn verwenden: https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html

In [ ]:
# Hier ist Platz für Ihre Lösung

### Aufgabe 7: Trainieren Sie das obige CNN-Modell nun mit dem kompletten Trainingsdatensatz. Wie hoch ist die Accuracy auf dem Testdatensatz?
##### Hinweis: Sie können diese Aufgabe auch bearbeiten, falls Sie die Aufgabe 6 nicht lösen konnten.

In [ ]:
# Hier ist Platz für Ihre Lösung

##### Wie wir sehen, ist die Vorhersagegenauigkeit des CNN-Modells bereits sehr hoch. Zur Veranschaulichung plotten wir ein zufälliges Bild aus dem Testdatensatz und schauen uns an, was die Vorhersage des Modells ist. Die Vorhersageergebnisse des CNNs stehen dabei für die Wahrscheinlichkeiten, die das Modell den Klassen (Zahlen von 0-9) zuordnet.

In [ ]:
# zufälliger Index für ein Bild aus dem Testdatensatz
image_index = 26

# plotte das Bild
pyplot.imshow(testX[image_index], cmap=pyplot.get_cmap('gray'))
pyplot.show()

# berechne die Vorhersage des CNNs
image = testX_cnn[image_index].reshape(1, 28, 28, 1)
digit = cnn.predict(image)
print("Vorhersage des CNNs: \n{}".format(digit[0]))

### Aufgabe 8: Können Sie die Vorhersagegenauigkeit noch weiter steigern? Welche Modellparameter lassen sich beim CNN dazu noch verändern? Experimentieren Sie.

In [ ]:
# Hier ist Platz für Ihre Lösung